In [ ]:
import os
import threading
import time
import tkinter.messagebox
from tkinter import *
from tkinter import filedialog
import sklearn as ss
print(ss.__version__)
from tkinter import ttk
from ttkthemes import themed_tk as tk

from mutagen.mp3 import MP3
from pygame import mixer
import real_time_video as rtv

root = tk.ThemedTk()
root.get_themes()                 # Returns a list of all themes that can be set
root.set_theme("radiance")         # Sets an available theme

# Fonts - Arial (corresponds to Helvetica), Courier New (Courier), Comic Sans MS, Fixedsys,
# MS Sans Serif, MS Serif, Symbol, System, Times New Roman (Times), and Verdana
#
# Styles - normal, bold, roman, italic, underline, and overstrike.

statusbar = ttk.Label(root, text="Welcome to Melody", relief=SUNKEN, anchor=W, font='Times 10 italic')
statusbar.pack(side=BOTTOM, fill=X)

# Create the menubar
menubar = Menu(root)
root.config(menu=menubar)

# Create the submenu

subMenu = Menu(menubar, tearoff=0)

playlist = []

from os import walk
# playlist - contains the full path + filename
# playlistbox - contains just the filename
# Fullpath + filename is required to play the music inside play_music load function

def browse_file():
    global filename_path
    label=get_emotion()
    #"angry" ,"disgust","scared", "happy", "sad", "surprised","neutral"
    if label=="neutral":
            print("Natural")
           # label1="natural"
    elif label=="angry"  or label=="disgust" :
            print("Angry")
            #lable1="angry"
    elif label=="happy" or label=="surprised" :
            print("Happy")
            #label1="happy"
    elif label=="sad":
            print("Sad")
            #label1="sad"
    elif label=="scared":
            print("Scared")
            #label1="sad"
    path="C:/Users/Khushboo Thakre/Desktop/Music-Player_Lyrics_Classification/playlist/"+label+"/"
    
    print(path)
    if label!="no":
        _, _, files = next(walk(path))
        print(path)
        print("::"+path+files[0])
    #filename_path = filedialog.askopenfilename()
    #for i in files:
        
    #print(filename_path)
    #for i in filenames:
        add_to_playlist(label,files)
        for i in files:
        
            mixer.music.queue(path+i)
            print("added")


def add_to_playlist(label,filename):
    print(filename)
    path="C:/Users/Khushboo Thakre/Desktop/Music-Player_Lyrics_Classification/playlist/"+label+"/"
    index = 0
    print("hello")
    for i in filename:
        
    
        playlistbox.insert(index, i)
        playlist.insert(index, path+i)
        index += 1


menubar.add_cascade(label="File", menu=subMenu)
subMenu.add_command(label="Open", command=browse_file)
subMenu.add_command(label="Exit", command=root.destroy)


def about_us():
    login()
    #tkinter.messagebox.showinfo('About Melody', 'This is a music player build using Python Tkinter by @attreyabhatt')

def login():
    import dashboard
subMenu = Menu(menubar, tearoff=0)
menubar.add_cascade(label="Lyrics Analysis", menu=subMenu)
subMenu.add_command(label="Lyrics", command=about_us)

mixer.init()  # initializing the mixer

root.title("Melody")
root.iconbitmap(r'images/melody.ico')

# Root Window - StatusBar, LeftFrame, RightFrame
# LeftFrame - The listbox (playlist)
# RightFrame - TopFrame,MiddleFrame and the BottomFrame

leftframe = Frame(root)
leftframe.pack(side=LEFT, padx=30, pady=30)

playlistbox = Listbox(leftframe)
playlistbox.pack()

addBtn = ttk.Button(leftframe, text="+ Add", command=browse_file)
addBtn.pack(side=LEFT)


def del_song():
    selected_song = playlistbox.curselection()
    selected_song = int(selected_song[0])
    playlistbox.delete(selected_song)
    playlist.pop(selected_song)


delBtn = ttk.Button(leftframe, text="- Del", command=del_song)
delBtn.pack(side=LEFT)

rightframe = Frame(root)
rightframe.pack(pady=30)

topframe = Frame(rightframe)
topframe.pack()

lengthlabel = ttk.Label(topframe, text='Total Length : --:--')
lengthlabel.pack(pady=5)

currenttimelabel = ttk.Label(topframe, text='Current Time : --:--', relief=GROOVE)
currenttimelabel.pack()


def show_details(play_song):
    file_data = os.path.splitext(play_song)

    if file_data[1] == '.mp3':
        audio = MP3(play_song)
        total_length = audio.info.length
    else:
        a = mixer.Sound(play_song)
        total_length = a.get_length()

    # div - total_length/60, mod - total_length % 60
    mins, secs = divmod(total_length, 60)
    mins = round(mins)
    secs = round(secs)
    timeformat = '{:02d}:{:02d}'.format(mins, secs)
    lengthlabel['text'] = "Total Length" + ' - ' + timeformat

    t1 = threading.Thread(target=start_count, args=(total_length,))
    t1.start()


def start_count(t):
    global paused
    # mixer.music.get_busy(): - Returns FALSE when we press the stop button (music stop playing)
    # Continue - Ignores all of the statements below it. We check if music is paused or not.
    current_time = 0
    while current_time <= t and mixer.music.get_busy():
        if paused:
            continue
        else:
            mins, secs = divmod(current_time, 60)
            mins = round(mins)
            secs = round(secs)
            timeformat = '{:02d}:{:02d}'.format(mins, secs)
            currenttimelabel['text'] = "Current Time" + ' - ' + timeformat
            time.sleep(1)
            current_time += 1


def play_music():
    global paused

    if paused:
        mixer.music.unpause()
        statusbar['text'] = "Music Resumed"
        paused = FALSE
    else:
        try:
            stop_music()
            time.sleep(1)
            selected_song = playlistbox.curselection()
            selected_song = int(selected_song[0])
            play_it = playlist[selected_song]
            mixer.music.load(play_it)
            mixer.music.play()
            statusbar['text'] = "Playing music" + ' - ' + os.path.basename(play_it)
            show_details(play_it)
        except:
            tkinter.messagebox.showerror('File not found', 'Melody could not find the file. Please check again.')


def stop_music():
    mixer.music.stop()
    statusbar['text'] = "Music Stopped"


paused = FALSE


def pause_music():
    global paused
    paused = TRUE
    mixer.music.pause()
    statusbar['text'] = "Music Paused"


def rewind_music():
    play_music()
    statusbar['text'] = "Music Rewinded"


def set_vol(val):
    volume = float(val) / 100
    mixer.music.set_volume(volume)
    # set_volume of mixer takes value only from 0 to 1. Example - 0, 0.1,0.55,0.54.0.99,1


muted = FALSE


def mute_music():
    global muted
    if muted:  # Unmute the music
        mixer.music.set_volume(0.7)
        volumeBtn.configure(image=volumePhoto)
        scale.set(70)
        muted = FALSE
    else:  # mute the music
        mixer.music.set_volume(0)
        volumeBtn.configure(image=mutePhoto)
        scale.set(0)
        muted = TRUE


middleframe = Frame(rightframe)
middleframe.pack(pady=30, padx=30)

playPhoto = PhotoImage(file='images/play.png')
playBtn = ttk.Button(middleframe, image=playPhoto, command=play_music)
playBtn.grid(row=0, column=0, padx=10)

stopPhoto = PhotoImage(file='images/stop.png')
stopBtn = ttk.Button(middleframe, image=stopPhoto, command=stop_music)
stopBtn.grid(row=0, column=1, padx=10)

pausePhoto = PhotoImage(file='images/pause.png')
pauseBtn = ttk.Button(middleframe, image=pausePhoto, command=pause_music)
pauseBtn.grid(row=0, column=2, padx=10)

# Bottom Frame for volume, rewind, mute etc.

bottomframe = Frame(rightframe)
bottomframe.pack()

rewindPhoto = PhotoImage(file='images/rewind.png')
rewindBtn = ttk.Button(bottomframe, image=rewindPhoto, command=rewind_music)
rewindBtn.grid(row=0, column=0)

mutePhoto = PhotoImage(file='images/mute.png')
volumePhoto = PhotoImage(file='images/volume.png')
volumeBtn = ttk.Button(bottomframe, image=volumePhoto, command=mute_music)
volumeBtn.grid(row=0, column=1)

scale = ttk.Scale(bottomframe, from_=0, to=100, orient=HORIZONTAL, command=set_vol)
scale.set(70)  # implement the default value of scale when music player starts
mixer.music.set_volume(0.7)
scale.grid(row=0, column=2, pady=15, padx=30)


def on_closing():
    stop_music()
    root.destroy()

def get_emotion():
    emotion=rtv.detect_emotion()

    return emotion
root.protocol("WM_DELETE_WINDOW", on_closing)
root.mainloop()

1.3.1
pygame 2.5.2 (SDL 2.28.3, Python 3.8.18)
Hello from the pygame community. https://www.pygame.org/contribute.html
2.13.0
2.13.1
1/1 [==============================] - 0s 406ms/step
neutral
Emotion Detected Successfully...
neutral
no
Not detected...
C:/Users/Khushboo Thakre/Desktop/Music-Player_Lyrics_Classification/playlist/no/
1.3.1


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Khushboo Thakre\anaconda3\envs\music\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Khushboo Thakre\AppData\Local\Temp\ipykernel_7008\2886099391.py", line 100, in about_us
    login()
  File "C:\Users\Khushboo Thakre\AppData\Local\Temp\ipykernel_7008\2886099391.py", line 104, in login
    import dashboard
  File "C:\Users\Khushboo Thakre\Downloads\Music-Player_Lyrics_Classification\dashboard.py", line 21, in <module>
    loaded_model = pickle.load(open(f4, 'rb'))
ModuleNotFoundError: No module named 'sklearn.svm.classes'


no
Not detected...
C:/Users/Khushboo Thakre/Desktop/Music-Player_Lyrics_Classification/playlist/no/
no
Not detected...
C:/Users/Khushboo Thakre/Desktop/Music-Player_Lyrics_Classification/playlist/no/
no
Not detected...
C:/Users/Khushboo Thakre/Desktop/Music-Player_Lyrics_Classification/playlist/no/
1/1 [==============================] - 0s 42ms/step
neutral
Emotion Detected Successfully...
neutral
Natural
C:/Users/Khushboo Thakre/Desktop/Music-Player_Lyrics_Classification/playlist/neutral/
C:/Users/Khushboo Thakre/Desktop/Music-Player_Lyrics_Classification/playlist/neutral/
::C:/Users/Khushboo Thakre/Desktop/Music-Player_Lyrics_Classification/playlist/neutral/atif.mp3
['atif.mp3', 'sajani.mp3']
hello
added
added
